<a href="https://colab.research.google.com/github/tvas20/imagem-que-vende/blob/main/GPT_imagem_que_vende.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================ 🔧 Instalar dependências ============================
!pip install -U openai onnxruntime gradio pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 786.8/786.8 kB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 52.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.7/59.7 MB 17.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.5/324.5 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 7.2 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.99.1
    Uninstalling openai-1.99.1:
      Successfully uninstalled openai-1.99.1
  Attempting uninstall: gradio-client
    Found existing installation: gradio_client 1.11.0
    Uninstalling gradio_client-1.11.0:
      Successfully uninstalled gradio_client-1.11.0
  Attempting uninstall: gradio
    Found existing installation: gradio 5.41.0
    Uninstalling gradio-5.41.0:
      Successfully uninstalled gradio-5.41.0


In [ ]:
# ============================ 📦 Imports ============================
import os
import io
import random
from PIL import Image
import gradio as gr
from openai import OpenAI
from google.colab import userdata
import base64

- Inserir abaixo o nome da sua chave secreta

In [ ]:
name_key = str(input("Digite o nome da sua chave aqui no classroom (não é o value): "))

Digite o nome da sua chave aqui no classroom (não é o value): GPT_KEY


In [ ]:
os.environ['OPENAI_API_KEY'] = userdata.get(f"{name_key}")
client = OpenAI()

In [ ]:
# ============================ 🧠 Edição de Imagem com OpenAI ============================
def gerar_imagem_otima(lista, gatilho, tipo, descricao, ambiente):
  gatilhos = {
    'Escassez': "iluminação dramática ou elementos visuais que criem tensão e foco imediato.",
    'Prova Social': "Sugira diretamente a presença de outros usuários (por exemplo algum ser humano fazendo o uso ou consumo do produto (o que for mais adequado para a situação e produto destacado)) ou a popularidade (como marcas de uso, o usuario fazendo o consumo ou uso do produto (o que for mais adequado ao produto principal da imagem), reflexos, sombras de pessoas, objetos duplicados).",
    'Autoridade': "Aplique estética limpa, cores sólidas e simetria para transmitir profissionalismo e confiança.",
    'Curiosidade': "Introduza zonas de sombra, partes propositalmente fora de foco ou parcialmente reveladas para criar mistério no produto.",
    'Exclusividade': "Use paletas de cores sofisticadas (preto, dourado, branco na ambientação), e destaque o objeto principal com iluminação direcionada.",
    'Progresso Visual': "Aplique leve repetição visual, marcas sutis de progresso ou sequências naturais na composição."
  }

  tipo_ambiente= {"automatico":'''
                    Inserir o alimento em ambientes comerciais compatíveis:

                    Hambúrguer → hamburguerias com estilo industrial, madeira, luz quente.

                    Açaí, sorvete, doces → sorveterias, açaiterias ou vitrines modernas, com fundo alegre, colorido e tropical.

                    Doces, bombons, bolos → confeitarias, vitrines ou mesas decoradas com estética gourmet

                    Sushi → restaurante japonês elegante com luz indireta, madeira clara ou escura, bambu e utensílios tradicionais.

                    Massas, carnes, pratos quentes → restaurantes acolhedores, iluminação amarelada e cenários de mesa posta.

                    Pizza, massas → cantina italiana, forno à lenha ao fundo, mesa rústica.

                    Cafés e sobremesas → cafés aconchegantes com luz natural, xícaras e pratos bem compostos sobre mesas de madeira.

                    Bebidas (café, coquetel, cerveja) → ambientes como cafeterias, bares ou festas, com luz ambiente adequada ao clima do produto.''',
                  "imagem base":"Inserir o alimento na ambientação da imagem original",
                  'gourmetizado interno':'''Um ambiente sofisticado e minimalista, com iluminação indireta estrategicamente posicionada, superfícies elegantes (mármore, vidro, inox), e organização impecável. Elementos como louças de design moderno, utensílios premium e fundos neutros reforçam o aspecto clean e refinado. A paleta de cores é contida, com contraste elegante e composição controlada. Tudo remete a luxo, exclusividade e atenção aos detalhes.''',
                  'gourmetizado externo':'''Um cenário externo cuidadosamente montado, como um piquenique gourmet ou evento ao ar livre elegante, com mesas de madeira clara ou estruturas modernas sob gazebos. Há arranjos florais sofisticados, louças brancas e utensílios premium. A luz natural é valorizada com filtros ou tecidos brancos esvoaçantes. A composição transmite leveza, exclusividade e estilo editorial, com um toque de natureza cuidadosamente estilizada.''',
                  'artesanal interno':'''Um espaço rústico e acolhedor, com paredes em tijolo aparente ou madeira envelhecida, iluminação amarelada e suave, e objetos feitos à mão distribuídos com naturalidade. Há móveis de madeira maciça, tecidos naturais como linho ou algodão cru, e utensílios artesanais à vista. A estética transmite conforto, tradição e conexão com o feito à mão. A luz entra por janelas pequenas, criando sombras suaves e um clima intimista.''',
                  'artesanal externo':'''Um cenário ao ar livre em meio à natureza, com mesas de madeira sob árvores, tecidos leves ao vento, cestarias, potes de cerâmica e flores silvestres espalhadas com casualidade. O chão pode ser de terra batida, pedras ou grama, com luz solar filtrada por folhagens. A composição evoca simplicidade, conexão com o natural e uma estética campestre autêntica.'''
                  }

  tipo_produto = {"Alimenticio":f'''Comidas e Bebidas:

                    O alimento deve estar em destaque absoluto, com cores vivas, texturas realçadas (gordura, cremosidade, crocância, frescor) e iluminação que gere apetite imediato.

                    Usar tons quentes (vermelho, amarelo, laranja) que ativam o gatilho da fome e impulsividade.

                    {tipo_ambiente[ambiente]}

                    ''',
                    "Produtos em geral":'''Eletrodomésticos (ex: geladeira, aspirador, micro-ondas)
                    → Inserir em cômodos residenciais realistas: cozinha moderna, lavanderia ou sala organizada. Usar iluminação natural e cores neutras para transmitir familiaridade e confiança.

                    Produtos de praia (ex: prancha de surf, óculos)
                    → Cenários litorâneos com sol forte, areia, mar, céu azul e vegetação tropical. Gatilhos de liberdade, emoção, verão e desejo.

                    Perfumes florais ou suaves
                    → Fundo etéreo com flores, tecidos leves, cores pastel, luz difusa. Remete a romantismo, feminilidade, leveza.

                    Perfumes amadeirados/orientais
                    → Ambientação com madeira, floresta, tons escuros, iluminação lateral dramática. Gatilhos de força, masculinidade, mistério, natureza.

                    Produtos esportivos (tênis, garrafas, relógios fitness)
                    → Ambientes de treino, rua, academia. Iluminação forte e direta, fundo urbano ou de ação. Ativa energia, superação, movimento.

                    Produtos de luxo (relógios, joias, carros)
                    → Fundo escuro com iluminação controlada, elementos metálicos ou nobres. Estética clean e contrastada. Gatilho de exclusividade e desejo aspiracional.

                    Eletrônicos (notebooks, celulares, fones)
                    → Escritórios modernos, setups gamer, ambientes tecnológicos com luz fria. Cores frias, metálicas ou neon. Evoca precisão, inovação e produtividade.

                    Cuidados pessoais e cosméticos
                    → Banheiros organizados, penteadeiras bem iluminadas, elementos de bem-estar (plantas, toalhas). Visual clean, suave, que transmite autoestima, frescor e autocuidado.'''}

  val_gatilhos = str()

  for i in gatilho:
    val_gatilhos += f"{gatilhos[i]}\n"

  prompt = f'''
  Edite a imagem existente (ou as imagens existentes) de forma realista, inserindo ou ajustando o produto principal em destaque dentro de uma ambientação que reforce sua funcionalidade, simbologia e apelo emocional, respeitando seu contexto de uso real.
  Gatilhos mentais:
  {val_gatilhos}

  O foco deve estar visualmente centrado no produto, com uso estratégico de profundidade de campo, iluminação e cores para conduzir o olhar. O estilo deve ser realista ou hiper-realista.

  Ambientação por Categoria de Produto:
  {tipo_produto[tipo]}

  Gatilhos Visuais Técnicos:
  Cores: Escolher conforme o efeito emocional desejado (ex: vermelho e amarelo para fome; azul para confiança; verde para saúde e assim por diante...).

  Saturação:

  Alta → Impacto, juventude, impulsividade.

  Baixa → Luxo, sofisticação, nostalgia.

  Brilho:

  Brilho alto → Limpeza, leveza, otimismo.

  Brilho baixo → Drama, elegância, intensidade emocional.

  Foco seletivo (bokeh): Usar para centralizar a atenção no produto.

  Iluminação:

  Difusa → Suavidade e conforto.

  Direcional/dura → Impacto e clareza.

  Sombras:

  Suaves → Realismo e naturalidade.

  Fortes → Profundidade e contraste.

  Restrições:
  Não adicionar textos como slogans, descrições ou chamadas.

  Apenas os textos presentes fisicamente no rótulo ou embalagem do produto devem ser visíveis.

  O estilo visual deve ser realista ou hiper-realista, com qualidade profissional para uso comercial.

  A marca e texto associados ao produto seja por meio de impressão ou gravados devem ser fielmente mantidos na imagem final.

  Descricao complementar  do produto: {descricao}
  '''
  lista_tratada = []

  try:
    imagem_produto = lista[0]
    imagem_produto1 = lista[1]

    buffer1 = io.BytesIO()
    imagem_produto1.save(buffer1, format='png')
    buffer1.seek(0)

    buffer1.name = "imagem1.png"

    lista_tratada.append(buffer1)
  except:
    imagem_produto = lista[0]

  buffer = io.BytesIO()
  imagem_produto.save(buffer, format='png')
  buffer.seek(0)
  buffer.name = "imagem.png"

  lista_tratada.append(buffer)

  result = client.images.edit(
    model="gpt-image-1",
    image=lista_tratada,
    prompt=prompt
  )

  image_base64 = result.data[0].b64_json
  image_bytes = base64.b64decode(image_base64)

  image_pil = Image.open(io.BytesIO(image_bytes)).convert('RGBA')

  return image_pil

In [ ]:
# ============================ 🧠 Geração de legenda com OpenAI ============================
def gerar_legenda(descricao, gatilho):
    prompt = f"""
Você é um redator de marketing. Crie uma legenda de Instagram persuasiva e criativa para o seguinte produto:

Caracteristicas: {descricao}
Gatilho psicológico a usar: {gatilho}

A legenda deve:
- Ter até 300 caracteres
- Usar emojis
- Incluir uma chamada para ação
- Gerar desejo de compra

Legenda:
"""
    response = client.chat.completions.create(
        model='gpt-4o',
        messages=[
            {"role": "system", "content": "Você é um redator especialista em marketing digital."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.9,
        max_tokens=150
    )
    return response.choices[0].message.content.strip()

In [ ]:
# ============================ 🧩 Função principal do sistema ============================
def gerar_post(descricao, pl_holder, gatilho, imagem, imagem1, pl_holder1, gatilho_imagem, tipo_produto, ambiente):
    if imagem is None:
        return None, "⚠️ Por favor, envie uma imagem do produto."

    lista = [imagem, imagem1]
    imagem_otima = gerar_imagem_otima(lista, gatilho_imagem, tipo_produto, descricao, ambiente)
    legenda = gerar_legenda(descricao, gatilho)

    return imagem_otima, legenda

In [ ]:
# ============================ 🌐 Interface Gradio ============================
gatilhos_0 = [
    "ESCASSEZ: Cria urgência e medo de perder uma oportunidade.",
    "PROVA SOCIAL: Mostra que outras pessoas já aprovaram ou usam o produto.",
    "AUTORIDADE: Transmite credibilidade e confiança por meio de sinais de profissionalismo.",
    "CURIOSIDADE: Estimula o interesse ao esconder parcialmente informações.",
    "EXCLUSIVIDADE: Faz o público se sentir especial ou parte de um grupo seleto.",
    "PROGRESSO VISUAL: Indica avanço ou evolução, incentivando continuidade."
]
gatilhos_1 = [
    "ESCASSEZ: Crie urgência com foco visual intenso.",
    "PROVA SOCIAL: Indique que outras pessoas estão envolvidas.",
    "AUTORIDADE: Use estética limpa e simétrica para confiança.",
    "CURIOSIDADE: Mostre só parte do produto para gerar mistério.",
    "EXCLUSIVIDADE: Destaque o produto com elegância e contraste.",
    "PROGRESSO VISUAL: Indique avanço com repetição ou sequência."
]
interface = gr.Interface(
    fn=gerar_post,
    inputs=[
        gr.Textbox(label="📝 Descrição do Produto [obrigatorio]", placeholder="Ex: Perfume arbo 100ml da perfumaria o boticario, preço normal: R$180,00; preço com desconto: R$ 98,79"),
        gr.Textbox(value=f"{gatilhos_0[0]}\n{gatilhos_0[1]}\n{gatilhos_0[2]}\n{gatilhos_0[3]}\n{gatilhos_0[4]}\n{gatilhos_0[5]}", interactive=False, label="Efeitos sobre a legenda:"),
        gr.Radio(["Escassez", "Prova Social", "Autoridade", "Curiosidade", "Exclusividade", "Progresso Visual"],label="🎯 Otimize a sua copy de vendas com um dos seguintes gatilhos:", info="Gatilhos Psicológicos"),
        gr.Image(label="📸 Imagem do Produto 1 [obrigatorio]", type="pil"),
        gr.Image(label="📸 Imagem do Produto 2 [opcional]", type="pil"),
        gr.Textbox(value=f"{gatilhos_1[0]}\n{gatilhos_1[1]}\n{gatilhos_1[2]}\n{gatilhos_1[3]}\n{gatilhos_1[4]}\n{gatilhos_1[5]}", interactive=False, label="Efeitos sobre a imagem:"),
        gr.CheckboxGroup(["Escassez", "Prova Social", "Autoridade", "Curiosidade", "Exclusividade", "Progresso Visual"], label="Otimize as imagens seguindo os seguintes gatilhos:", info="Gatilhos Psicologicos [opcional]"),
        gr.Radio(["Alimenticio","Produtos em geral"], label="Tipo do produto:", info="Escolha o tipo: [obrigatorio]"),
        gr.Radio(["automatico","imagem base","artesanal externo","artesanal interno","gourmetizado interno","gourmetizado externo"], label="Ambientação:", info="Escolha o tipo: [obrigatorio]",value="automatico")
        ],
    outputs=[
        gr.Image(label="🖼️ Imagem Otimizada"),
        gr.Textbox(label="📢 Copy de vendas")
    ],
    title="📸 Self Sell 🚀",
    description="Envie a imagem de um produto, a descrição e o tipo de gatilho. A IA removerá o fundo, criará um novo visual e escreverá uma legenda chamativa."
)

In [ ]:
# ============================ 🚀 Lançar aplicação ============================
interface.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://83077eb73c7c2b3b51.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Using existing dataset file at: .gradio/flagged/dataset1.csv
